In [1]:
import logging
from typing import Callable, Literal, Optional, Union

import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.optimize import minimize

from bwlogger import StyleAdapter

In [2]:


from utils import (
    calculate_portfolio_var,
    calculate_risk_contribution, 
    risk_budget_objective,
    total_weight_constraint,
    optmize_risk_budget,
    calculate_weights
)
w_clip = pd.Series(
    {
        0: 0.21965866188326,
        1: 0.547691984160487,
        2: 0.132499791033892,
        3: 0.10015056292236,
    }
)
cov = pd.DataFrame(
    {
        0: {0: 0.04, 1: 0.002, 2: 0.00625, 3: 0.072},
        1: {0: 0.008, 1: 0.01, 2: 0.03125, 3: 0.018},
        2: {0: 0.004, 1: 0.005, 2: 0.0625, 3: 0.018},
        3: {0: 0.032, 1: 0.002, 2: 0.0125, 3: 0.09},
    }
)
rc_t = pd.Series([1 / 4] * 4, index=cov.index)
w0 = [0, 1, 0, 0]
x_t = [
    0.25,
    0.25,
    0.25,
    0.25,
]

In [4]:
calculate_weights("IV", cov, pd.Series(x_t))

0    0.164354
1    0.657414
2    0.105186
3    0.073046
dtype: float64

In [5]:
optmize_risk_budget(cov, pd.Series(x_t), pd.Series(w0), tol=1e-20)

0    0.219658
1    0.547691
2    0.132500
3    0.100150
dtype: float64

In [ ]:
def _calculate_portfolio_var(w: pd.Series, cov: pd.DataFrame) -> float:
    return w.T @ cov @ w


def _calculate_risk_contribution(w: pd.Series, cov: pd.DataFrame) -> pd.Series:
    vol = _calculate_portfolio_var(w, cov)**0.5
    mrc = cov @ w
    return mrc * w.T / vol


def _risk_budget_objective(
    x: pd.Series, risk_pct: pd.Series, cov: pd.DataFrame
) -> float:
    vol = _calculate_portfolio_var(x, cov)**0.5
    rc_t = vol * risk_pct

    rc = _calculate_risk_contribution(x, cov)
    return np.square(rc - rc_t).sum()


def _total_weight_constraint(w: pd.Series) -> float:
    return w.sum() - 1.0


def _optmize_risk_budget(
    cov: pd.DataFrame,
    rc_t: Optional[pd.Series] = None,
    w0: Optional[pd.Series] = None,
    cons: Optional[list[dict[str, Callable[pd.Series, pd.Series]]]] = [],
    **params,
) -> pd.Series:
    n = len(cov.index)
    if w0 is None:
        w0 = pd.Series([1] * n, index=cov.index) / n
    if rc_t is None:
        rc_t = pd.Series([1] * n, index=cov.index) / n
    cons_sum_1 = {"type": "eq", "fun": _total_weight_constraint}
    cons = tuple(cons + [cons_sum_1])

    res = minimize(
        _risk_budget_objective,
        w0,
        args=(rc_t, cov),
        constraints=cons,
        **params,
    )
    return pd.Series(res.x, index=cov.index)


_optmize_risk_budget(cov, pd.Series(x_t), pd.Series(w0), tol=1e-20).values

In [ ]:
def calculate_portfolio_var(w, V):
    # function that calculates portfolio risk
    w = np.matrix(w)
    return (w * V * w.T)[0, 0]


def calculate_risk_contribution(w, V):
    # function that calculates asset contribution to total risk
    w = np.matrix(w)
    sigma = np.sqrt(calculate_portfolio_var(w, V))
    # Marginal Risk Contribution
    MRC = V * w.T
    # Risk Contribution
    RC = np.multiply(MRC, w.T) / sigma
    return RC


def risk_budget_objective(x, pars):
    # calculate portfolio risk
    V = pars[0]  # covariance table
    x_t = pars[1]  # risk target in percent of portfolio risk
    sig_p = np.sqrt(calculate_portfolio_var(x, V))  # portfolio sigma
    risk_target = np.asmatrix(np.multiply(sig_p, x_t))
    asset_RC = calculate_risk_contribution(x, V)
    J = sum(np.square(asset_RC - risk_target.T))[0, 0]  # sum of squared error
    return J


def total_weight_constraint(x):
    return np.sum(x) - 1.0


def long_only_constraint(x):
    return x


cons = (
    {"type": "eq", "fun": total_weight_constraint},
    {"type": "ineq", "fun": long_only_constraint},
)
res = minimize(
    risk_budget_objective,
    w0,
    args=[cov.values.tolist(), x_t],
    method="SLSQP",
    constraints=cons,
    options={"disp": True},
    tol=1e-20,
)
w_rb = np.asmatrix(res.x)
w_rb

In [ ]:
_calculate_portfolio_var(w_clip, cov) - calculate_portfolio_var(w_clip.values, cov.values)

In [ ]:
_calculate_risk_contribution(w_clip, cov)
# calculate_risk_contribution(w_clip.values, cov.values).tolist()

In [ ]:
x = w_clip
V = cov
sig_p = np.sqrt(calculate_portfolio_var(w_clip.values, cov.values))  # portfolio sigma
risk_target = np.asmatrix(np.multiply(sig_p, [1/4]*4))
risk_target

In [ ]:
var = _calculate_portfolio_var(w_clip, cov)
rc_t = var * pd.Series([1 / 4] * 4)
rc_t